In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## Data completeness

In [5]:
df_clicks_train = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/influence_data/df_clicks_train_unique_idea_indexed.parquet')

In [6]:
df_clicks_train.head()

,session,aid,ts,session_encoded,aid_encoded,res_encoded,res_encoded_tfidf,pred,pred_tfidf
0,0,1582117,1.660245e+12,0,666553,"[281161, 775436, 516320, 764100, 451673, 29874...","[516320, 281161, 775436, 764100, 298740, 59310...","[667924.0, 1840418.0, 1226444.0, 1813509.0, 10...","[1226444.0, 667924.0, 1840418.0, 1813509.0, 70..."
1,1,959544,1.659991e+12,1,404059,"[730713, 90511, 360049, 302753, 21022, 44560, ...","[730713, 90511, 360049, 21022, 302753, 44560, ...","[1734061.0, 215311.0, 854637.0, 718983.0, 5004...","[1734061.0, 215311.0, 854637.0, 50049.0, 71898..."
2,3,618373,1.660060e+12,2,260158,"[414678, 737002, 45682, 739209, 567885, 558347...","[412613, 347944, 353885, 243317, 739209, 27765...","[984459.0, 1748862.0, 108125.0, 1754057.0, 134...","[979687.0, 825711.0, 840023.0, 578649.0, 17540..."
3,6,179011,1.659997e+12,3,75355,"[408325, 772989, 366503, 103911, 45251, 781573...","[408325, 103911, 366503, 45251, 276711, 560040...","[969717.0, 1834442.0, 870131.0, 247477.0, 1070...","[969717.0, 247477.0, 870131.0, 107068.0, 65719..."
4,7,129869,1.660222e+12,4,54768,"[54768, 233739, 40339, 549466, 295711, 466644,...","[54768, 233739, 253269, 40339, 295711, 549466,...","[129869.0, 555948.0, 95488.0, 1304914.0, 70243...","[129869.0, 555948.0, 602191.0, 95488.0, 702437..."


In [12]:
df_clicks_train.to_parquet("c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_clicks_train_for_colab")

In [15]:
df_clicks_train_unique = df_clicks_train.copy()

In [16]:
df_clicks_train_unique['pred'] = df_clicks_train_unique['res_encoded'].apply(clicks_aid_encoder.inverse_transform)

In [ ]:
df_clicks_train_unique['pred_tfidf'] = df_clicks_train_unique['res_encoded_tfidf'].apply(clicks_aid_encoder.inverse_transform)

In [ ]:
df_clicks_train_unique.to_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_clicks_train_indexed')

In [2]:
df_clicks_train = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/reindexed_data/idea_data/df_clicks_train_indexed')

In [7]:
def set_ranks(df: pd.DataFrame):
    df_c = df.copy()
    df_c["rank"] = df_c.groupby("session")["ts"].rank(method="first", ascending=True)
    return df_c[["session", "aid", "rank"]]

In [8]:
df_clicks_r = set_ranks(df_clicks_train)
#df_carts_r = set_ranks(df_carts_t)
#df_orders_r = set_ranks(df_orders_t)

In [9]:
def rank_stat(df_r: pd.DataFrame, ks: list):
    df_mr = df_r.groupby("session").agg("max")["rank"]
    for k in ks:
        cnt_me = len(df_mr[df_mr >= k])
        print(f"Users with more than {k} test items: {cnt_me} ({round(cnt_me / len(df_mr) * 100, 2)}%)")

In [10]:
rank_stat(df_clicks_r, [3, 5, 10, 20])

Users with more than 3 test items: 0 (0.0%)
Users with more than 5 test items: 0 (0.0%)
Users with more than 10 test items: 0 (0.0%)
Users with more than 20 test items: 0 (0.0%)


In [11]:
#rank_stat(df_carts_r, [3, 5, 10, 20])

In [12]:
#rank_stat(df_orders_r, [3, 5, 10, 20])

# Metrics

In [13]:
K_RECS = 200
K_TEST = 5

In [14]:
df_clicks_r = pd.read_parquet('c:/Users/tanni/Desktop/rec_sys/Project/prepared_dataset_Nikita/clicks/test')

In [16]:
#df_carts_r = pd.read_parquet("data/carts/")

In [17]:
#df_orders_r = pd.read_parquet("data/orders/")

In [15]:
def extract_n_elements(row, n):
    return np.pad(row[:n], (0, max(n - len(row), 0)), mode='constant', constant_values=-1)


In [16]:
def prepare_data(df_test, df_recs, k_test, k_recs):
    valid_session = np.intersect1d(
        df_test["session"].unique(),
        df_recs["session"].unique()
    )

    df_test_l = set_ranks(df_test)
    df_test_l = df_test_l[df_test_l["session"].isin(valid_session)]
    df_test_l = df_test_l[df_test_l["rank"] <= k_test]
    df_test_l = df_test_l.groupby("session")["aid"].apply(list).reset_index().sort_values("session")

    df_test_l["aid"] = (
        df_test_l["aid"]
        .apply(lambda x: np.pad(
            x[:k_test],
            (0, max(k_test - len(x), 0)),
            mode="constant",
            constant_values=-1
        ))
    )
    test = np.stack(df_test_l["aid"].values, axis=0)

    df_recs_l = df_recs[df_recs["session"].isin(valid_session)]
    df_recs_l["modified_pred"] = (
        df_recs_l["pred"]
        .apply(lambda x: extract_n_elements(x, k_recs))
    )
    
    recs = df_recs_l["modified_pred"].tolist()

    return test, np.array(recs)


In [17]:
test_clicks, recs_clicks = prepare_data(df_clicks_r, df_clicks_train, K_TEST, K_RECS)

C:\Users\tanni\AppData\Local\Temp\ipykernel_328\1135287424.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (


In [19]:
rows_to_remove = np.where(np.any(test_clicks == -1, axis=1))[0]

# Удаляем соответствующие строки из обеих матриц
test_clicks = np.delete(test_clicks, rows_to_remove, axis=0)
recs_clicks = np.delete(recs_clicks, rows_to_remove, axis=0)

In [21]:
recs_clicks.shape

(121194, 200)

In [20]:
import numpy as np


class Metrics:
    def __init__(
            self,
            k: int,
            recommendations: np.ndarray,
            holdout: np.ndarray
            ):
        self.k = k
        self.recommendations = recommendations[:, :k]
        self.holdout = holdout

    def report(self):
        print(f"HR@{self.k} = {round(self.hit_rate(), 4)}")
        print(f"MRR@{self.k} = {round(self.mean_reciprocal_rank(), 4)}")
        print(f"Recall@{self.k} = {round(self.recall(), 4)}")

    def hit_rate(self):
        return (
            self._get_hit_mask()
            .any(axis=1)
            .mean()
        )

    def mean_reciprocal_rank(self):
        hits_mask = self._get_hit_mask()

        idx = np.argwhere(hits_mask.argmax(axis=1)).squeeze(axis=1)

        return np.sum(
            1 / (hits_mask[idx].argmax(axis=1) + 1)
        ) / hits_mask.shape[0]

    def recall(self):
        hits_mask = self._get_hit_mask()
        return (
            hits_mask.sum(axis=1) / self.recommendations.shape[1]
        ).sum() / self.recommendations.shape[0]

    def _get_hit_mask(self) -> np.ndarray:
        return (
            self.holdout[..., None] ==
            np.expand_dims(self.recommendations, axis=1)
        ).any(axis=1)

In [21]:
#from rec_base import Metrics
metrics_carts = Metrics(K_RECS, np.array(recs_clicks), test_clicks)
print(metrics_carts.report())

HR@200 = 0.4394
MRR@200 = 0.0108
Recall@200 = 0.25
None


# Normalize

In [22]:
def prepare_data(df_test, df_recs, k_test, k_recs):
    valid_session = np.intersect1d(
        df_test["session"].unique(),
        df_recs["session"].unique()
    )

    df_test_l = set_ranks(df_test)
    df_test_l = df_test_l[df_test_l["session"].isin(valid_session)]
    df_test_l = df_test_l[df_test_l["rank"] <= k_test]
    df_test_l = df_test_l.groupby("session")["aid"].apply(list).reset_index().sort_values("session")

    df_test_l["aid"] = (
        df_test_l["aid"]
        .apply(lambda x: np.pad(
            x[:k_test],
            (0, max(k_test - len(x), 0)),
            mode="constant",
            constant_values=-1
        ))
    )
    test = np.stack(df_test_l["aid"].values, axis=0)
    print(test.shape)

    df_recs_l = df_recs[df_recs["session"].isin(valid_session)]
    df_recs_l["modified_pred"] = (
        df_recs_l["pred_tfidf"]
        .apply(lambda x: extract_n_elements(x, k_recs))
    )
    
    recs = df_recs_l["modified_pred"].tolist()

    return test, recs


In [23]:
test_clicks, recs_clicks = prepare_data(df_clicks_r, df_clicks_train, K_TEST, K_RECS)

(213464, 5)


C:\Users\tanni\AppData\Local\Temp\ipykernel_328\2324496305.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recs_l["modified_pred"] = (


In [38]:
np.array(recs_clicks).shape

(213272, 200)

In [24]:
metrics_carts = Metrics(K_RECS, np.array(recs_clicks), test_clicks)
print(metrics_carts.report())

HR@200 = 0.4372
MRR@200 = 0.0081
Recall@200 = 0.242
None
